In [215]:
import numpy as np
import pandas as pd

from skimage.io import imread

In [72]:
class ObjectWrapper(object):
    def __init__(self, s):
        self.s = s
        self.type = type(s[0])

In [323]:
def check_all_elements_have_the_same_property(array, func):
    if len(array) == 0:
        return True, None
    try:
        first_element_type = func(array[0])
    except:
        return True, None
    do_all_have_property = all(func(x) == first_element_type
                               for x in array)

    return do_all_have_property, first_element_type


class ObjectWrapper(object):
    def __init__(self, obj):
        self.obj = obj
        self.type = type(obj)


class MultiSeries(pd.Series):
    _metadata = ['data_type']

    @property
    def _constructor(self):
        return MultiSeries

    @property
    def _constructor_expanddim(self):
        return MultiDataFrame
        
    def __init__(self, *args, **kwargs):
        super(MultiSeries, self).__init__(*args, **kwargs)
        
        data = kwargs.get('data')
        if data is None:
            data = args[0]
        
        check_result, data_type = check_all_elements_have_the_same_property(data, type)
        if not check_result:
            raise ValueError('Not all elements the same type')
                
        if data_type is not None:
            self.data_type = data_type
        else:            
            self.data_type = type(data._values[0])
            
            
class MultiDataFrame(pd.DataFrame):
    @property
    def _constructor(self):
        return MultiDataFrame

    @property
    def _constructor_sliced(self):
        return MultiSeries
    
    
    def __init__(self, *args, **kwargs):
        data = kwargs.get('data')
        if data is None:
            data = args[0]
        
        data_to_check = []
        if isinstance(data, list):
            data_to_check = data
        elif isinstance(data, dict):
            data_to_check = data.values()
        
        for d in data_to_check:
            if not isinstance(d, MultiSeries):
                raise ValueError('All data must be MultiSeries instances')            
        super(MultiDataFrame, self).__init__(*args, **kwargs)
        
        
    def get_columns_of_type(self, column_type):
        columns_to_select = [
            col_name for col_name in self.columns
            if self[col_name].data_type == column_type
        ]
        
        return self[columns_to_select]        

In [352]:
s1 = MultiSeries([pd.Series([1, 2, 3], index=['a', 'b', 'c']),
                   pd.Series([4, 5, 6], index=['d', 'e', 'g'])])
s2 = MultiSeries([1, 2, 3])
s3 = MultiSeries([{"k1": "v1"}, {"k2": 'v2'}])
s4 = MultiSeries([imread('/Users/iwitaly/Downloads/1.jpg')])


df = MultiDataFrame({
    'first': s1,
    'second': s2,
    'third': s3,
    'fouth': s4
})

# df = MultiDataFrame([
#        s1, s2, s3, s4
# ])
# s = df['third']

print(
    df['second'].data_type
)

<class 'numpy.int64'>


In [351]:
isinstance(1, np.int64)

False